# Retrieval Results Analysis

The goal of this notebok is to investigate the quality of the retrieval when using RAG and the Datamodels re-ranking  
To be able to analyse this we will see how each method ranks the golden documents for 50 test samples and how different the ranking were to similar and different performance situations  
Finally, we will further oberserve the special scenarios where the baseline performed well but neither the methods acheived the expected answer

## Setup

In [11]:
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import json
import os

In [4]:
## Load wiki
wiki = pl.read_ipc("../../data/wiki_dump2018_nq_open/processed/wiki.feather")
wiki.head()

text,title
str,str
"""Aaron Aaron ( or ; ""Ahärôn"") i…","""Aaron"""
"""God at Sinai granted Aaron the…","""Aaron"""
"""his rod turn into a snake. The…","""Aaron"""
"""however, Aaron and Hur remaine…","""Aaron"""
"""Aaron and his sons to the prie…","""Aaron"""


In [3]:
## Read test df
test_df  = pl.read_ipc("previews/50_test.feather")
test_df.head()

example_id,question,answers,text,idx_gold_in_corpus
i64,str,list[str],str,i64
-5266947057771511513,"""who was the first woman to win…","[""Judith Cynthia Aline Keppel""]","""Judith Cynthia Aline Keppel (b…",21033303
-7256500086799415081,"""who carried florida by 537 vot…","[""George W. Bush"", ""Bush""]","""United States presidential ele…",20988218
7104554727863075462,"""who's the original singer of h…","[""Kris Kristofferson""]","""'' Help Me Make It Through The…",20972472
-5172636572480122331,"""when did the subway open in ne…","[""October 27 , 1904"", ""1904""]","""The New York City Subway is a …",20995376
-7121343730302515106,"""when did the passion of the ch…","[""2004"", ""February 25 , 2004""]","""The Passion of the Christ ( al…",21033680


In [5]:
## Example of gold passage based on the last sample showed in the previous cell
wiki[21033680]

text,title
str,str
"""The Passion of the Christ ( al…","""The Passion of the Christ"""


In [6]:
#### Load generations results
PREFIX_PATH = "previews"

previews = {
    "0": "preview_50_L2",
    "1": "preview_50_IP",
    "2": "new_prompt_preview_50_cosine",
    "4": "new_prompt_preview_50_L2"
}

dfs = []

for f in os.listdir("previews_results"):

    _preview_num = f.split("_")[1]


    if f.endswith("datamodels.feather"):
        retrieval = "datamodels"
        type = previews[_preview_num]

    elif f.endswith("baseline.feather"):
        retrieval = "baseline"
        type = "baseline"

    else:
        retrieval = "rag"
        type = previews[_preview_num]

    _df = pl.read_ipc(f"previews_results/{f}")
    _df = _df.with_columns(pl.lit(retrieval).alias("retrieval"))
    _df = _df.with_columns(pl.lit(type).alias("type"))
    dfs.append(_df)

In [15]:
rag_retrievals = {
    "idx": [],
    "retrieval": [],
    "type": [],
    "docs": []
}

for dir in os.listdir("previews_results"):
    if dir in previews.values():

        docs = json.load(open(f"previews_results/{dir}/retrieval/rag_retrieval_indexes.json", "r"))
        for _key in docs.keys():

            rag_retrievals["idx"].append(_key)
            rag_retrievals["retrieval"].append("rag")
            rag_retrievals["type"].append(dir)
            rag_retrievals["docs"].append(docs[_key][:4])